## Deliverable 2. Create a Customer Travel Destinations Map.

In [59]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [60]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Albany,42.6001,-73.9662,25.66,46,16,1.01,US,2022-02-06 17:06:01,few clouds
1,1,Bluff,-46.6000,168.3333,39.79,81,1,5.12,NZ,2022-02-06 17:07:16,clear sky
2,2,Arraial Do Cabo,-22.9661,-42.0278,87.66,58,0,18.41,BR,2022-02-06 17:07:16,clear sky
3,3,Hermanus,-34.4187,19.2345,73.90,79,7,9.53,ZA,2022-02-06 17:07:17,clear sky
4,4,Ruteng,-8.6136,120.4721,63.07,91,45,1.72,ID,2022-02-06 17:07:17,scattered clouds


In [61]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [62]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
2,2,Arraial Do Cabo,-22.9661,-42.0278,87.66,58,0,18.41,BR,2022-02-06 17:07:16,clear sky
6,6,Samarai,-10.6167,150.6667,81.30,80,100,12.75,PG,2022-02-06 17:07:18,overcast clouds
10,10,Kavieng,-2.5744,150.7967,81.41,78,100,4.12,PG,2022-02-06 17:07:19,overcast clouds
11,11,Rikitea,-23.1203,-134.9692,78.78,71,91,16.08,PF,2022-02-06 17:07:19,overcast clouds
16,16,Kuching,1.5500,110.3333,75.42,100,40,3.44,MY,2022-02-06 17:07:22,moderate rain
25,25,Faanui,-16.4833,-151.7500,81.63,75,96,20.92,PF,2022-02-06 17:07:24,light rain
28,28,Formosa,-26.1775,-58.1781,85.98,37,20,18.41,AR,2022-02-06 17:07:25,few clouds
31,31,Bredasdorp,-34.5322,20.0403,79.56,37,25,9.86,ZA,2022-02-06 17:05:34,scattered clouds
33,33,Atuona,-9.8000,-139.0333,78.67,74,10,20.47,PF,2022-02-06 17:07:26,clear sky
36,36,Santa Maria,-29.6842,-53.8069,84.42,30,7,11.68,BR,2022-02-06 17:07:27,clear sky


In [63]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                206
City                   206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Country                203
Date                   206
Current Description    206
dtype: int64

In [64]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [65]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"].copy()]

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

C:\Users\mleegan\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,87.66,clear sky,-22.9661,-42.0278,
6,Samarai,PG,81.30,overcast clouds,-10.6167,150.6667,
10,Kavieng,PG,81.41,overcast clouds,-2.5744,150.7967,
11,Rikitea,PF,78.78,overcast clouds,-23.1203,-134.9692,
16,Kuching,MY,75.42,moderate rain,1.5500,110.3333,
25,Faanui,PF,81.63,light rain,-16.4833,-151.7500,
28,Formosa,AR,85.98,few clouds,-26.1775,-58.1781,
31,Bredasdorp,ZA,79.56,scattered clouds,-34.5322,20.0403,
33,Atuona,PF,78.67,clear sky,-9.8000,-139.0333,
36,Santa Maria,BR,84.42,clear sky,-29.6842,-53.8069,


In [66]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

C:\Users\mleegan\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [68]:
# 7. Drop the rows where there is no Hotel Name.
completed_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
completed_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,87.66,clear sky,-22.9661,-42.0278,Pousada Porto Praia
6,Samarai,PG,81.30,overcast clouds,-10.6167,150.6667,Nuli Sapi
10,Kavieng,PG,81.41,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
11,Rikitea,PF,78.78,overcast clouds,-23.1203,-134.9692,People ThankYou
16,Kuching,MY,75.42,moderate rain,1.5500,110.3333,The Ranee Boutique Suites


In [69]:
# 8a. Create the output File (CSV)
output_data_file = 'Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
completed_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in completed_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = completed_hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))